## The File for generating HTML tags

In [15]:
from wand.image import Image as wi
import os; from io import BytesIO; import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import re

from tesserocr import PyTessBaseAPI, RIL, iterate_level, PT, OEM, Justification

In [2]:
directory = "..\\datafiles\\"

In [3]:
def pdf2folderimages (dir_path, file):
    newfolderpath = dir_path + file.split('.')[0]
    #print(newfolderpath)
    try:
        new_dir_cmd = "mkdir "+ newfolderpath
        os.system(new_dir_cmd)
        #print(new_dir_cmd)
        sys_cmd = "convert -units PixelsPerInch -density 300 " + dir_path+file +" "+ newfolderpath+"\page.png"
        os.system(sys_cmd)
        #print(sys_cmd)
        return newfolderpath
    except:
        return False

In [ ]:
start = time.time()
FOLDERpath = pdf2folderimages(directory, "doc129.pdf")
tmp_time = time.time() - start
print(tmp_time)

In [5]:
print("Average Time:\t\t", tmp_time/129)

Average Time:		 2.0454631239868872


In [17]:
pytestapi_path = "C:\\Tesseract\\Tesseract-OCR-v5\\tessdata"

### OCR SCAN
* Images to Text Conversion 
* Table Detection (Table detection does happens in the java code but doesn't go in the grain)
     * TH , TR , TD
* Background Detection
    * Specific Line Background
    * Word Level, Block Level & Line Level (Under Progress)
    
* Font Characterstics
    * Font Density ( Tried to do in jar but was inefficent, insufficent and inaccurate)
        * Bold 
        * Light
    * Font Size
* Hand Written Detection (ITS UNDER PROGRESS)
    * HandWritten Model vs Digital Text Model
    * Evaluation of Metrices
    
     
      

In [11]:
def line_bg_detection(bbox, numBIN, NUMPYimage):
    if bbox:
        x1, y1, x2, y2 = bbox
        colorbin = np.ones((numBIN, 3), dtype = 'int16') *-1

        combs = np.where(NUMPYimage[y1:y2, x1:x2, 0] != -1) # Check if the image didn't had the -1 there
        if (combs[0].size != 0): # check if the size is not zero 
            bin_i = 0
            # Choose randomly bins (20 is the default set right now )
            # To check the image or the bin or whatsoever 
            for i in np.random.choice(range(combs[0].shape[0]), numBIN):
                x = combs[1][i]
                y = combs[0][i]
                colorbin[bin_i] = NUMPYimage[y1+y, x1+x, :]
                bin_i += 1
            # Historgram 
            hist = {}
            for r, g, b in colorbin:
                if not (r == -1):
                    RGB = str(r)+"_"+str(g)+"_"+str(b)
                    if RGB in list(hist.keys()):
                        hist[RGB] += 1
                    else:
                        hist[RGB] = 1
            if len(list(hist.keys())):
                return list(hist.keys())[list(hist.values()).index(max(list(hist.values())))].split("_") ## RGB value in list
    return 0

In [12]:
def get_table_row_tags(api, bbox):
    ri = api.GetIterator()
    level = RIL.TEXTLINE
    ri = api.GetIterator()
    
    buffer = 0
    
    lines = "<table>\n"
    
    for rl in iterate_level(ri, level):
        if rl:
            tmp = rl.GetUTF8Text(level)
            if (tmp.rstrip()):
                c_x1, c_y1, c_x2, c_y2  = rl.BoundingBox(level)
                if (c_y1 >= (bbox[1] - buffer) and c_y2 <= (bbox[3] + buffer)):
                    lines += "<tr>" + rl.GetUTF8Text(level) + "\n</tr>\n"
                if (c_y2 > (bbox[3] + buffer)):
                    break
    lines += "</table>\n"
    # print(lines)
    return lines

In [13]:
def sorted_nicely(l):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

In [22]:
def line_bgColorPreProcess(PILimage, tessAPI):
    if (PILimage.mode != 'RGB'): #check if image not in RGB then make it one: for bg color detection
        NUMPYimage = np.array(PILimage.convert('RGB'), dtype='int16')
    else:
        NUMPYimage = np.array(PILimage, dtype='int16')

    ri = tessAPI.GetIterator()
    level = RIL.WORD
    # Word by word iterator 
    for r in iterate_level(ri, level):
        if r:
            bbox = r.BoundingBox(level)
            if bbox: ## Now black (-1) 'em out
                NUMPYimage[bbox[1]:bbox[3], bbox[0]:bbox[2], :] = -1
    return NUMPYimage

In [29]:
def generate_HTML (FOLDERpath):
    '''
    params:
    FOLDERpath --> path of directoray which contain all the MR document images
    returns:
    HTML format string containing result of OCR scan    
    '''
    
    FileTextBuffer = ""
    FileTextBuffer += "<document>\n"

    filesList = os.listdir(FOLDERpath)
    
    pagecounter  = 1
    for filename in sorted_nicely(filesList):
        start = time.time()
        page = Image.open(FOLDERpath+"\\"+filename) #readimage in PIL format
        print("MR chart '", FOLDERpath+"\\"+filename, "' is under-process")
        
        thisPageText = ""
        thisPageText += "<page" +str(pagecounter)+">\n"

        with PyTessBaseAPI(path = pytestapi_path) as api:
            api.SetImage(page)
            api.Recognize()
            
            ## Preprosessing for background detection --START
            NUMPYimage = line_bgColorPreProcess(page, api)
            ## Preprosessing for background detection --END

            ri = api.GetIterator()
            level = RIL.BLOCK # Block based Values 

            # Table is detectable ( we can get the verticle and horizental Lins )
            for r in iterate_level(ri, level):
                if r:
                    block_type = r.BlockType() # Type of that specific block
                    # print(r.GetUTF8Text(level))
                    just = r.ParagraphInfo()[0]
                    #print(r.GetUTF8Text(level), " ### ", just, " ******", just == Justification.RIGHT)
                    
                    if (block_type == PT.TABLE):
                        img = r.GetBinaryImage(level)
                        # img.show() to show cropped table image
                        text_rows = get_table_row_tags(api, r.BoundingBox(level))
                        thisPageText += text_rows
                    elif(block_type not in [PT.UNKNOWN, PT.FLOWING_IMAGE, PT.HEADING_IMAGE, PT.PULLOUT_IMAGE, PT.HORZ_LINE, PT.VERT_LINE, PT.NOISE]):
                        ###  Backgrpound Color algo -START Params (bbox, numBIN, NumpyImage)
                        rgb_result = line_bg_detection(r.BoundingBox(level), 20, NUMPYimage)
                        ###  Backgrpound Color algo -END
                        if (rgb_result):
                            thisPageText += "<text font_bg = '" + rgb_result[0] + "," + rgb_result[1] + "," + rgb_result[2] +"'>\n" + r.GetUTF8Text(level) + "</text>\n"
                        else:
                            thisPageText += "<text>\n" + r.GetUTF8Text(level) + "</text>\n"

        FileTextBuffer += thisPageText + "</page" +str(pagecounter)+">\n"
        pagecounter += 1
        print("Page Time:\t\t", time.time() - start)
        
    FileTextBuffer += "</document>\n"
    
    return FileTextBuffer

In [30]:
FOLDERpath = '..\\datafiles\\doc22'

In [31]:
COST = time.time()
GeneratedText = generate_HTML(FOLDERpath)
tmp_time = time.time() - COST
print("Time on Document:\t", tmp_time)

MR chart ' ..\datafiles\doc22\doc22-0.png ' is under-process
Page Time:		 3.8349900245666504
MR chart ' ..\datafiles\doc22\doc22-1.png ' is under-process
Page Time:		 2.2131128311157227
MR chart ' ..\datafiles\doc22\doc22-2.png ' is under-process
Page Time:		 3.6932380199432373
MR chart ' ..\datafiles\doc22\doc22-3.png ' is under-process
Page Time:		 3.156672954559326
MR chart ' ..\datafiles\doc22\doc22-4.png ' is under-process
Page Time:		 2.9767675399780273
MR chart ' ..\datafiles\doc22\doc22-5.png ' is under-process
Page Time:		 2.474260091781616
MR chart ' ..\datafiles\doc22\doc22-6.png ' is under-process
Page Time:		 2.138235330581665
MR chart ' ..\datafiles\doc22\doc22-7.png ' is under-process
Page Time:		 2.7827718257904053
MR chart ' ..\datafiles\doc22\doc22-8.png ' is under-process
Page Time:		 2.9533467292785645
MR chart ' ..\datafiles\doc22\doc22-9.png ' is under-process
Page Time:		 2.1311888694763184
MR chart ' ..\datafiles\doc22\doc22-10.png ' is under-process
Page Time:	

In [15]:
print("Average time:\t\t", tmp_time/52)

Average time:		 2.667533966211172


In [20]:
print(GeneratedText)

<document>
<page1>
<text font_bg = '255,255,255'>
Page 1 of 11

</text>
<text font_bg = '255,255,255'>
mae
cet #:

From Visit On: 04/11/16
Office Visit

</text>
<text font_bg = '244,250,255'>
4/11/2016 Pe | MRN: 1388862

</text>
<text font_bg = '255,255,255'>
Reason for Visit
Heme/Onc Care

</text>
<table>
<tr>New Patient


</tr>
<tr>Reason for Visit History

</tr>
<tr>AVS Reports


</tr>
<tr>Date/Time Report Action User


</tr>
<tr>4/11/2016 12:09 After Visit Summary Printed Michelle Norris


</tr>
<tr>PM


</tr>
<tr>Most recent update: 4/11/2016 10:44 AM by


</tr>
<tr>Vital Signs Erica Newton


</tr>
<tr>BP Pulse Temp(Src) Resp Ht Wt


</tr>
<tr>428/102 98 36.9 °C (98.4 °F) 20 185.4 cm (72.99") 73.483 kg (162 Ib)


</tr>
<tr>mmHg (Oral)


</tr>
<tr>BMI SpO2


</tr>
</table>
<text font_bg = '255,255,255'>
21.38 kg/m2 94%

</text>
<text font_bg = '255,255,255'>
Progress Notes
Sherin Fetouh, MD at 04/11/16 1040
Status: Signed
Date of Service: 4/11/2016

</text>
<text>
Subjective:

</te

In [46]:
with open('doc22-text8.xml', 'w+') as f:
    f.write(GeneratedText)